# Продолжим про ансамбли

Основная идея: у нас есть **слабые алгоритмы** (переученные или недоученные) и мы хотим их как-то собрать воедино и тем самым усилить наше решение:
$$b_1(x), b_2(x), ... , b_k(x)$$

1. Бэггинг (для деревьев) - усредняем переученные глубокие деревья решения, при этом деревья должны быть различны (за счет бутстрапа), желательно максимально независимо:
$$a_k(x) = \dfrac{1}{k}\sum_{i=1}^k b_i(x)$$
2. Бустинг (сейчас мы тут)
3. Войтинг
4. Стекинг

# Идея бустинга

Необходимо решить задачу $\{x_i \to y_i\}_{i=1}^m$

1. Возьмем простые базовые модели
2. Будем строить композицию **последовательно**
3. Каждая следующая модель будет строиться так, чтобы максимально корректировоть ошибку ранее сделанного шага.

***Аналогия: бэггинг это обучения в студентов в группе, бустинг - обучение команды под решение задачи, где каждый в чем-то силен, а в чем-то нет.***

   Основная причина применения  бустинга заключается в последовательном достижении оптимума для алгорится за меньшее число шагов, чем при бэггинге, одно тут плохо с параллелизацией. Следовательно (забегаая вперед), количество делевьев для бустинга является гиперпараметром.

 Итак, мы будем искать неосредненный (как ранее в беггинге алгоритм):
$$a_k(x) = \sum_{j=1}^k b_s(x)$$
где $k$ - количество деревьев.

Сделаем первый шаг:
$$\sum_{i=1}^m L(y_i, b_1(x_i)) \to \underset{b_1(x)}{\min}$$

Тогда для $m$-ой модели:
$$\dfrac{1}{m}\sum_{i=1}^m L(y_i, a_{N-1}(x_i)+b_N(x_i)) \to \underset{b_N(x)}{\min},$$
то есть для $a_{N-1}(x_i)$ уже обученной модели мы добавляем $b_N(x)$, что бы ее прогноз уменьшил уже существующею ошибку.

Как же строить модель $b_N$, что бы она улучшила итоговый результат построения модели $a_N$  по сравнению $a_{N-1}$? 

## Самый простой вариант бустинга для понимания

Рассмотрим самый простой вариант нашего бустинга на квадратичной ошибке:
$$\dfrac{1}{m}\sum_{i=1}^m (a_{N-1}(x_i)+b_N(x_i)-y_i)^2 \to \underset{b_N(x)}{\min},$$

Тогда первый шаг:
$$\dfrac{1}{m}\sum_{i=1}^m (b_1(x_i)-y_i)^2 \to \underset{b_1(x)}{\min},$$

Тогда второй шаг:
$$\dfrac{1}{m}\sum_{i=1}^m \left(b_1(x_i) + b_2(x_i)-y_i)\right)^2 \to \underset{b_2(x)}{\min} \Rightarrow$$
$$\dfrac{1}{m}\sum_{i=1}^m \left(b_2(x_i)-(y_i-b_1(x_i))\right)^2 \to \underset{b_2(x)}{\min},$$
то есть вторая модель $b_2(x_i)$ обучается на новой целевой переменной, не на $y_i$, а на разнице между $y_i-b_1(x_i)$.
Если вторая модель $b_2(x_i)$ идельно компенсирует разницу, то получим нулевую ошибку.

Третий шаг: 
$$\dfrac{1}{m}\sum_{i=1}^m \left(b_2(x_i)-(y_i-b_1(x_i)-b_2(x_i))\right)^2 \to \underset{b_3(x)}{\min},$$

В итоге мы за маленькое количество шагов, можем найти прекрасное решение,  
но есть **маленькая проблемка** - переобучение ...

## Сложности с функцией потерь

Давайте посмотрим на **"любую"** функцию потерь:
$$\dfrac{1}{m}\sum_{i=1}^m L(y_i, a_{N-1}(x_i)+b_N(x_i)) \to \underset{b_N(x)}{\min}$$

Расмотрим логистическую регрессию, в качестве примера:
$$a_N(x)=\text{sign} \sum_{i=1}^N b_n(x)$$
$$\dfrac{1}{m}\sum_{i=1}^m L(y,z) = \log(1+\exp(-y z))$$


Попробуем найти остатки как в MSE:
$$\sum_{i=1}^m L(y,z) = \log(1+\exp(-(y_i-a_{N-1}(x_i))b_N(x_i) )) \to \underset{b_N(x)}{\min}$$
где $y$ это $y_i-a_{N-1}(x_i)$, а $z$ - $b_N(x_i)$ тогда получим, что:
- если построенная ранее модель хорошо отработала, то он не участвует в обучении $y_i = a_N(x_i)$ и получили 0 в произведении.
- если построенная ранее модель плохо отработала, то $y_i-a_{N-1}(x_i)=\pm 2$. Ок. Ну ладно , давайте теперь расмотрим **только** для ответов с ошибками:

$$\sum_{i=1}^m L(y,z) = \log\left(1+\exp\left(-\left(\dfrac{y_i-a_{N-1}(x_i)}{2}b_N(x_i)\right) \right)\right) \to \underset{b_N(x)}{\min}$$
Если $y_i \neq a_N(x_i)$, то модель учится искать корректный класс, однако ...

- $y_i=+1,\;\;\;  b_N(x_i)= -0.5$(это класс -1), нам нужно $b_N(x_i)>0.5$ - корректура маленькая
- $y_i=+1, \;\;\; b_N(x_i)= -100$(это класс -1), нам нужно $b_N(x_i)>100$ - корректура большая,
а наша модель не знает насколько нужно скорректироваться и тут все стало плохо ...

Поэтому нам нужен другой подход.

## Градиентный спуск

Ошибка на $x_i$ при прогнозе новой модели, равном $z_i$:
$$\sum_{i=1}^m L(y_i, a_{N}(x_i)+z_i)$$

Посчитаем производную:
$s_i^{(m)} = -\dfrac{\partial}{\partial z}L(y_i,z_i)\Big|_{z = a_{N}(x_i)}$, тогда
- знак показывает сторону сдвига прогноза $x_i$
- величина показывает как сильно двигаться для уменьшения ошибки
- если ошибка меньше порога, то менять что-либо смысла нет.

  Тогда мы построили прекрасную модель машинного обучения будущего:

$$\dfrac{1}{m}\sum_{i=1}^m \left(b_N(x_i)-s_i^{(m)}\right)^2 \to \underset{b_N(x)}{\min}$$
где $s_i^{(m)} = -\dfrac{\partial}{\partial z}L(y_i,z)\Big|_{z = a_{N-1}(x_i)}$


### Градиентный бустинг MSE

$$s_i^{(N)} = -\dfrac{\partial}{\partial z}L(y_i,z_i)\Big|_{z = a_{N-1}(x_i)} = -\dfrac{1\partial}{2\partial z}(z-y^2)\Big|_{z = a_{N-1}(x_i)}=$$
$$= -(a_N(x_i)-y_i)=y_i-a_N(x_i)$$

что это напоминает ...

$$\sum_{i=1}^m L(b_N(x_i) - (y_i - a_{N-1}(x_i)))^2 \to \underset{b_N(x)}{\min},$$

И это очень просто и бысто для MSE:  
$y_i = 10, \;\; a_{N-1}(x_i)=5$ тогда $s_i = 5$    
$y_i = 10, \;\; a_{N-1}(x_i)=-5$ тогда $s_i = -15$ 

### Градиентный бустинг логистической функции потерь

$$s_i^{(m)} = -\dfrac{\partial}{\partial z}L(y_i,z_i)\Big|_{z = a_{N-1}(x_i)} = -\dfrac{\partial}{\partial z}
\log(1+\exp(-y_i z))\Big|_{z = a_{N-1}(x_i)}=$$
$$=\dfrac{y_i}{1+\exp(y_i a_{N-1}(x_i))}$$

$$\dfrac{1}{m}\sum_{i=1}^m L\left(b_N(x_i) - \dfrac{y_i}{1+\exp(y_i a_{N-1}(x_i))}\right)^2 \to \underset{b_N(x)}{\min}$$

$y_i$ и $a_{N-1}(x_i)$ значительно больше 0:   $\dfrac{y_i}{1+\exp(y_i a_{N-1}(x_i))} \to 0$    
разница $y_i$ и $a_{N-1}(x_i)$ большая отрицательная:   $\dfrac{y_i}{1+\exp(y_i a_{N-1}(x_i))} \to \pm 1$ 

# Гиперпараметры бустнинга

В ходе экспериментов с моделями установлено:
1. Градиентный бустинг уменьшает смешение моделей
2. Разброс может увеличиваться

Следствие: 
- нельзя брать глубокие модели, обычно глубиной 2-5 `max_depth`.
- нужно подбирать число деревьев `n_estimators`

Важные, но влияют чуть меньше:

3. простые модели (базовые) очень простые, они могут или медленно и слишком быстро шагать, поэтому нужно регулировать шаг `learning_rate`: $a_N = a_{N-1}(x_i) + \eta\; b_N(x_i))$, $\eta \in (0, +\infty)$
4. рандомизация по признакам `subsample` в интервале $(0.0, 1.0]$ (обычно замедляюд шаг и увеличивают рандомизацию), при рандомизации можно счить OOB-ошибки. Обычно увеличивает смешение и уменьшает дисперсию.

Важно помнить, что длинна шага коррелирует с количеством деревьев!

Важно запомнить - НИКАКИХ РЕШЕТОК!!!